In [12]:
from PIL import Image
import requests
import pandas as pd
import os
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPProcessor, CLIPModel
import torch
import torch.nn as nn
#import torchvision.transforms as transforms


In [24]:
from torchvision.transforms import ToTensor

ImportError: cannot import name 'get_ctx' from 'torch.library' (c:\Users\sunny\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\library.py)

In [2]:
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [17]:
class LookingWhileListeningDataset(Dataset):
    def __init__(self, image_folder, pair_csv, transform=None, text_transform=None, k=2):
        self.image_folder = image_folder
        self.pairs = pd.read_csv(pair_csv)
        # If no transform is provided, use ToTensor as default
        self.transform = transform if transform is not None else transforms.ToTensor()
        self.text_transform = text_transform
        self.k = k

    def __getitem__(self, idx):
        row = self.pairs.iloc[idx]
        images, texts = [], []
        for i in range(self.k):
            image_filename = f"{row[f'stim{i+1}']}.jpg"
            image_path = os.path.join(self.image_folder, image_filename)
            image = Image.open(image_path)
            image = self.transform(image)  # This will convert the PIL image to a tensor
            images.append(image)

            text = row[f'stim{i+1}']
            if self.text_transform:
                text = self.text_transform(text)
            texts.append(text)
        return images, texts




In [5]:
class MultimodalModel(nn.Module):
    def __init__(self, encoder, text_encoder, fusion_module, k=2):
        super(MultimodalModel, self).__init__()
        self.encoder = encoder  # For images
        self.text_encoder = text_encoder  # For text
        self.fusion_module = fusion_module
        self.k = k

    def forward(self, images, texts):
        image_encodings = [self.encoder(image) for image in images]
        if self.text_encoder:
            text_encodings = [self.text_encoder(text) for text in texts]
        else:
            text_encodings = [None] * self.k
        fused_encodings = [self.fusion_module(image_enc, text_enc) for image_enc, text_enc in zip(image_encodings, text_encodings)]
        return fused_encodings

In [6]:

image_folder = "C:\\Users\\sunny\\Desktop\\research\\cogeval\\tablet_images\\images\\"

pair_csv = "C:\\Users\\sunny\\Desktop\\research\\cogeval\\lwl_trials_full.csv"




In [13]:
dataset = LookingWhileListeningDataset(
    image_folder= image_folder,
    pair_csv= pair_csv,
    transform=None, 
    text_transform=None,
    k=2
)

In [14]:
data_loader = DataLoader(dataset, batch_size=32, shuffle=False)

In [20]:
def calculate_softmax_probs(data_loader, processor, model):
    model.eval()  
    softmax_probs = []

    with torch.no_grad():  
        for batch in data_loader:
            images, texts = batch
            # Flatten the list of texts if they are in tuples for each image
            texts = [text for text_pair in texts for text in text_pair]
            # Process the images and texts through the CLIP processor
            # Convert the list of PIL Images to a list and then to a batch of tensors
            inputs = processor(text=texts, images=images, return_tensors="pt", padding=True)
            outputs = model(**inputs)
            logits_per_image = outputs.logits_per_image  # Image-text similarity score
            probs = logits_per_image.softmax(dim=1)  # Softmax to get label probabilities
            softmax_probs.extend(probs)
    
    # Convert the list of softmax probabilities to a tensor
    return torch.stack(softmax_probs)

In [21]:
softmax_probs = calculate_softmax_probs(data_loader, processor, model)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\sunny\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\_utils\collate.py", line 145, in collate
    else:
          
  File "c:\Users\sunny\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\_utils\collate.py", line 145, in <listcomp>
    else:
          
  File "c:\Users\sunny\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\_utils\collate.py", line 150, in collate
    return [collate(samples, collate_fn_map=collate_fn_map) for samples in transposed]
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.JpegImagePlugin.JpegImageFile'>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\sunny\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactivesh

In [7]:
# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])



NameError: name 'transforms' is not defined